## Solving bidomain equations on rectangular domain

### Importing required libraries

In [ ]:
%cd ../../..
from src.models.dynamics_models import *
from src.models.cell_models import *
from src.utils import *
import ufl

### Defining a domain, cell model and dynamics model and visualizing initial data

In [ ]:
domain = rectangle(1, 1, 32, 32)
x = ufl.SpatialCoordinate(domain)

In [ ]:
class CellModel(ReparametrizedFitzHughNagumo):
    def apply_current(self):
        locator = lambda x: x[0] < 0.1
        cells = fem.locate_dofs_geometrical(self.V1, locator)
        self.I_app.x.array[cells] = np.full_like(cells, 10)
        self.I_app_duration = 20

In [ ]:
class Model(BidomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -85

    def conductivity(self):
        # Muscle fibres
        self.fibers = ufl.as_vector([1, 0])

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IN - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EN - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

In [ ]:
cell_model = CellModel(domain)
model = Model(domain, cell_model)

In [ ]:
cell_model.plot_I_app(camera_direction='xy')

### Solving equations with given parameters

In [ ]:
model.solve(
    T=500,
    steps=500,
    save_to="rectangle/bidomain_MFN.mp4",
    signal_point=[0.5, 0.5, 0.0],
    camera_direction='xy',
    checkpoints=[10,20,30,40,50],
    checkpoint_file='figures/rectangle/bidomain_MFN'
)

### Plotting a fiber orientations

In [ ]:
plot_vector_field(domain, lambda x: (1, 0, 0), 0.03)

### Plotting a final plot of transmembrane potential

In [ ]:
plot_function(model.V_m_n, "V_m", show_mesh=False)

### Plotting a transmembrane potential in a given signal point

In [ ]:
model.plot_signal()